In [ ]:
# Gyakran változtatott konstansok

# Stop word-ök használata
USE_STOPWORDS = True

# Stemmelés használata
USE_STEM = True

In [ ]:
# Importok

import re

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sn
from nltk import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
# Konstansok

# Random seed-ek értéke
SHUFFLE_RANDOM_STATE = 42
TRAIN_RANDOM_STATE = 42
SGD_RANDOM_STATE = 42

# A program számára fontos fejlécek értéke az adatbázisban
TEXT = 'Sentence'
Y_HEADER = 'LABEL'

# Az adatbázisban található értékek megfeleltetése a programban
LABELS = {
    "NEG": 0,
    "SEM": 1,
    "POZ": 2
}

In [ ]:
# Szöveg feldolgozására alkalmas metódusok

URL_RE = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}([-a-zA-Z0-9()@:%_+.~#?&/=]*)'
WHITELIST_RE = '[^a-zA-Z0-9íÍöÖüÜóÓőŐúÚáÁéÉűŰ]'

def cleanse(i):
    """
    Adatbázisban szereplő dokumentumok megtisztítása az URL-ektől és a nem-alfanumerikus karakterektől.

    Parameters
    ----------
    i: int
        A tisztítandó dokumentumnak az indexe.

    Returns
    -------
    str
        A megtisztított dokumentum.
    """

    text = dataset[TEXT].iloc[i]
    text = re.sub(URL_RE, ' ', text)
    text = re.sub(WHITELIST_RE, ' ', text)
    text = ' '.join([text.lower()])
    return re.sub(' +', ' ', text)

def delete_empty_rows(dataset):
    """
    Az adatbázist megtisztítja az üres soroktól.

    Parameters
    ----------
    dataset: pandas.DataFrame
        Az adatbázisból készített DataFrame.

    Returns
    -------
    pandas.DataFrame
        A megtisztított adatbázis.
    """

    ids_to_delete = dataset.index[dataset[TEXT] == ' '].tolist()
    return dataset.drop(ids_to_delete)

In [ ]:

# Adatok előkészítése

dataset = pd.read_csv('db/sport_or_e_sport.csv', sep=';', error_bad_lines=False)
dataset = shuffle(dataset, random_state=SHUFFLE_RANDOM_STATE)
dataset.info()

dataset = delete_empty_rows(dataset)
for i  in range(len(dataset.index)):
    dataset[TEXT].iloc[i] = cleanse(i)

dataset = delete_empty_rows(dataset)
dataset.head(5)
X = dataset[TEXT].values
y = dataset[Y_HEADER].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=TRAIN_RANDOM_STATE)

In [ ]:
# Adatok vizualizálása

def plot_label_counts(y, title='y labels'):
    """
    Adatok eloszlásának ábrázolása.

    Parameters
    ----------
    y: numpy.ndarray
        Az adathalmaz címkéi, amelyre a vizualizálás történik.
    title: str
        Az elkészített ábra címe.
    """

    unique, counts = np.unique(y, return_counts=True)
    b = dict(zip(unique, counts))
    b_values = [b['SPORT'], b['VIDEÓJÁTÉK']]
    plt.barh(range(len(b)), b_values, align='center', color=['lightblue', 'lightgreen'])
    y_values = ["Sport", "Videójáték"]
    y_axis = np.arange(0, 2, 1)
    plt.yticks(y_axis, y_values)
    plt.title(title)
    plt.xlabel('Number of Samples in training Set')
    plt.ylabel('Label')
    ax = plt.gca()
    for i, v in enumerate(b_values):
        plt.text(ax.get_xlim()[1]/100, i, str(v), color='blue', fontweight='bold')
    plt.show()

plot_label_counts(y_train, 'Train eloszlas')
plot_label_counts(y_test, 'Test eloszlas')

In [ ]:
def evaluate(predict, labels):
    """
    A modell kiértékelése. Kiíratja az osztályozási eredményeket, a pontosságot, illetve az igazságmátrixot.

    Parameters
    ----------
    predict: numpy.ndarray
        A modell predikcióit tartalmazza a teszt adathalmazra.
    y: numpy.ndarray
        A teszt adathalmaz címkéit tartalmazza.
    """

    print('Classification report:')
    print(classification_report(labels, predict))
    print('Accuracy:')
    print(accuracy_score(labels, predict))

    print('Confusion matrix:')
    df_cm = pd.DataFrame(confusion_matrix(labels, predict, labels=['SPORT', 'VIDEÓJÁTÉK']),
                         index=[i for i in ['SPORT', 'VIDEÓJÁTÉK']],
                         columns=[i for i in ['SPORT', 'VIDEÓJÁTÉK']])
    plt.figure(figsize=(10,7))
    hm = sn.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
    hm.set(ylabel='True label', xlabel='Predicted label')
    plt.show()

In [ ]:
# Stop word-ök letöltése

nltk.download('stopwords')

In [ ]:
# Korpuszok létrehozása a dokumentumokból

all_stopwords = []

def create_corpus_by_dataset(sentences):
    """
    Korpusz létrehozása. Stop word-ök és stemmelés alkalmazása, amennyiben azok engedélyezve vannak.

    Parameters
    ----------
    sentences: list of numpy.ndarray
        Az adathalmazban található dokumentumokat tartalmazó lista, amelyen a stop word-ök alkalmazva lesznek.

    Returns
    -------
    list of str
        Az elkészített korpusz.
    """

    global all_stopwords
    corpus = []
    for sen in sentences:
        if USE_STOPWORDS:
            all_stopwords = stopwords.words('hungarian')
            whitelist = ["ne", "nem", "se", "sem"]
            sentence = [word for word in sen.split() if (word not in all_stopwords or word in whitelist)
                 and len(word) > 1]
        if USE_STEM:
            ps = SnowballStemmer('hungarian')
            sentence = [ps.stem(word) for word in sentence]
        sentence = ' '.join(sentence)
        corpus.append(sentence)

    return corpus

X_train_clean = create_corpus_by_dataset(X_train)
X_test_clean = create_corpus_by_dataset(X_test)

In [ ]:
# Bag of words modell létrehozása és illesztése

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.002, ngram_range=(1,3))
x_train_model = cv.fit_transform(X_train_clean)

In [ ]:
# SGD osztályozó létrehozása és illesztése

from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(random_state=SGD_RANDOM_STATE, learning_rate='constant',
                           eta0=0.01)
classifier.fit(x_train_model, y_train)

In [ ]:
# Beállított modellek kiértékelése

x_dev_model = cv.transform(X_test_clean)

dev_predict = classifier.predict(x_dev_model)

evaluate(dev_predict, y_test)

In [ ]:
# Grid Search előkészítése és illesztése

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
           ('vect', cv),
           ('clf', classifier)
])

parameters_to_tune = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (0.001, 0.002, 0.005, 0.01),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1,3), (1,4)),
    'clf__max_iter': (500, 1000, 2000, 5000),
    'clf__eta0': (0.005, 0.01, 0.02, 0.05)
}
clf=GridSearchCV(pipeline,parameters_to_tune, n_jobs=-1, verbose=1)
clf.fit(X_train_clean, y_train)

In [ ]:
# Optimális hiperparaméterekkel beállított modellek részleteinek kiírása

print(clf.best_estimator_)

In [ ]:
# Predikálás az optimális hiperparaméterekkel beállított modelleken

pred2 = clf.predict(X_test_clean)
evaluate(pred2, y_test)